In [1]:
import pandas as pd
import numpy  as np

from scipy   import sparse
from sklearn.feature_extraction.text import TfidfVectorizer

import re
import nltk
from nltk import *
nltk.download('stopwords')

from adarank import AdaRank

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lucas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


ModuleNotFoundError: No module named 'metrics'

In [ ]:
def process(text):
    
    if pd.isna(text):
        return ' - '
        
    text = text.lower() + ' '

    text = text.replace('/' , ' / ')
    text = text.replace('.' , ' . ')
    text = text.replace(',' , ' , ')
    text = text.replace('-' , ' - ')
    text = text.replace('^' , ' ^ ')

    text = re.sub(r'[  ]', r' ', text)
    
    text = text.replace('ser ' , 'serum ')
    text = text.replace('plas ', 'plasma ')
    text = text.replace('bld ' , 'blood ')
    text = text.replace('fld ' , 'fluid ')
    text = text.replace('synv ', 'synovial ')
    text = text.replace('plr ' , 'pleural ')
    text = text.replace('bpu ' , 'blood product unit ')

    text = [word for word in text.split() if word not in corpus.stopwords.words('english')]
    return ' '.join([word for word in text])

In [ ]:
queries = ["glucose in blood", "bilirubin in plasma", "white blood cells count"]

df = pd.read_excel("base_data.xlsx")[['Query id','f0','f1','f2','f3','Y']]
df.head()

,Query id,f0,f1,f2,f3,Y
0,1,Indirect antiglobulin test complement specific...,Indirect antiglobulin test complement specific...,Ser / Plas,ACnc,0
1,1,Rh Type in Blood,Rh,Bld,Type,2
2,1,Major crossmatch interpretation,Major crossmatch,Ser / Plas,Imp,0
3,1,Methicillin resistant Staphylococcus aureus Pr...,Staphylococcus aureus methicillin resistant is...,XXX,ACnc,0
4,1,Bilirubin total Mass / volume in Synovial fluid,Bilirubin,Synv fld,MCnc,0


In [ ]:
df['f0'] = df['f0'].apply(lambda f: process(f))
df['f1'] = df['f1'].apply(lambda f: process(f))
df['f2'] = df['f2'].apply(lambda f: process(f))
df['f3'] = df['f3'].apply(lambda f: process(f))
df.head()

,Query id,f0,f1,f2,f3,Y
0,1,indirect antiglobulin test complement specific...,indirect antiglobulin test complement specific...,serum / plasma,acnc,0
1,1,rh type blood,rh,blood,type,2
2,1,major crossmatch interpretation,major crossmatch,serum / plasma,imp,0
3,1,methicillin resistant staphylococcus aureus pr...,staphylococcus aureus methicillin resistant is...,xxx,acnc,0
4,1,bilirubin total mass / volume synovial fluid,bilirubin,synovial fluid,mcnc,0


In [ ]:
def get_l_norm(doc, query_word_ids):
    doc_l_norm = 0
    
    for word_id in query_word_ids:
        word_tfidf  = doc[(0,word_id)]
        doc_l_norm += pow(word_tfidf,2)
        
    doc_l_norm = np.sqrt(doc_l_norm)
    return doc_l_norm

def normalize_vector(doc, query_word_ids):
    l_norm = get_l_norm(doc, query_word_ids)
    return [doc[(0,word_id)] / l_norm if l_norm else 0 for word_id in query_word_ids]

def get_doc_cos_score(doc, query):
    return sum([doc[i] * query[i] for i in range(len(doc))]) 

In [ ]:
for i in range(1, len(queries) + 1):

    query = queries[i-1]

    tfidf_f0 = TfidfVectorizer()
    tfidf_f1 = TfidfVectorizer()
    tfidf_f2 = TfidfVectorizer()
    tfidf_f3 = TfidfVectorizer()
    
    vectorized_f0 = tfidf_f0.fit_transform(df[df["Query id"] == i]['f0'])
    vectorized_f1 = tfidf_f1.fit_transform(df[df["Query id"] == i]['f1'])
    vectorized_f2 = tfidf_f2.fit_transform(df[df["Query id"] == i]['f2'])
    vectorized_f3 = tfidf_f3.fit_transform(df[df["Query id"] == i]['f3'])

    vectorized_features = [vectorized_f0, vectorized_f1, vectorized_f2, vectorized_f3]

    query_f0 = tfidf_f0.transform([query])
    query_f1 = tfidf_f1.transform([query])
    query_f2 = tfidf_f2.transform([query])
    query_f3 = tfidf_f3.transform([query])

    vectorized_query_per_feature = [query_f0, query_f1, query_f2, query_f3]

    for vectorized_feature, feature_vectorized_query, feature_id in zip(vectorized_features, vectorized_query_per_feature, ["f0","f1","f2","f3"]):
        query_word_ids   = feature_vectorized_query.indices
        norm_query       = normalize_vector(feature_vectorized_query, query_word_ids)
        documents_scores = [get_doc_cos_score(normalize_vector(vectorized_feature[doc_id], query_word_ids), norm_query) for doc_id in range(0,vectorized_features[0].shape[0])]
        
        df.loc[df["Query id"] == i, feature_id] = documents_scores

df.head()

,Query id,f0,f1,f2,f3,Y
0,1,0.0,0.0,0.0,0,0
1,1,0.57069,0.0,1.0,0,2
2,1,0.0,0.0,0.0,0,0
3,1,0.0,0.0,0.0,0,0
4,1,0.0,0.0,0.0,0,0


In [ ]:
df["f0"] = pd.to_numeric(df["f0"])
df["f1"] = pd.to_numeric(df["f1"])
df["f2"] = pd.to_numeric(df["f2"])
df["f3"] = pd.to_numeric(df["f3"])

In [ ]:
from sklearn.model_selection import train_test_split

x1, x1_te, y1, y1_te =  train_test_split(df[df["Query id"]==1][["Query id", "f0", "f1", "f2", "f3"]], df[df["Query id"]==1]["Y"], test_size = .15)
x2, x2_te, y2, y2_te =  train_test_split(df[df["Query id"]==2][["Query id", "f0", "f1", "f2", "f3"]], df[df["Query id"]==2]["Y"], test_size = .15)
x3, x3_te, y3, y3_te =  train_test_split(df[df["Query id"]==3][["Query id", "f0", "f1", "f2", "f3"]], df[df["Query id"]==3]["Y"], test_size = .15)


In [ ]:
x = pd.concat([x1,x2,x3], axis=0)
x_te = pd.concat([x1_te,x2_te,x3_te], axis=0) 
y = pd.concat([y1,y2,y3], axis=0) 
y_te = pd.concat([y1_te,y2_te,y3_te], axis=0) 

In [ ]:
qid = np.array([1]*len(x1) + [2]*len(x2) + [3]*len(x3))
y   = np.array(y)
x_sparse = sparse.csr_matrix(np.array(x[["f0", "f1", "f2", "f3"]])) 

In [ ]:
model = AdaRank(verbose=True)
model.fit(x_sparse,y,qid)

1	1.5975334468125855	0	[1.         1.         0.76388928]	train 0.9213	valid 0.9213
2	1.518651954346032	0	[1.         1.         0.76388928]	train 0.9213	valid 0.9213


AdaRank(scorer=<metrics.NDCGScorer object at 0x00000111671C59A0>, verbose=True)

In [ ]:
qid_t = np.array([1]*len(x1_te) + [2]*len(x2_te) + [3]*len(x3_te))
y_te  = np.array(y_te)
x_te_sparse = sparse.csr_matrix(np.array(x_te[["f0", "f1", "f2", "f3"]])) 

In [ ]:
pred = model.predict(x_te_sparse, None)
print("The NDGC for each of the queries is respectively:",model.evaluate(y_te, pred, qid_t))

The NDGC for each of the queries is respectively: [1.         0.82490082 0.73060786]


In [ ]:
mae_general = 0
n_general   = 0

mae_values = [0,0,0,0,0]
n_values   = [0,0,0,0,0]

for real_val, pred_val in zip(y_te, pred):
    mae_values[real_val] += abs(real_val - pred_val)
    n_values[real_val]   += 1

for i, (mae, n) in enumerate(zip(mae_values, n_values)):
    print("The MAE for queries with rank",i,"is:")
    if n == 0:
        print("No test queries with that rank")
    else:
        print(mae / n)
        mae_general += mae / n
        n_general   += 1
    
print("The overall MAE is:")
print(mae_general/n_general)

The MAE for queries with rank 0 is:
0.0
The MAE for queries with rank 1 is:
1.0
The MAE for queries with rank 2 is:
1.040753575043954
The MAE for queries with rank 3 is:
3.0
The MAE for queries with rank 4 is:
2.4024665531874145
The overall MAE is:
1.4886440256462738


In [ ]:
mse_general = 0
n_general   = 0

mse_values = [0,0,0,0,0]
n_values   = [0,0,0,0,0]

for real_val, pred_val in zip(y_te, pred):
    mse_values[real_val] += pow(real_val - pred_val, 2)
    n_values[real_val]   += 1

for i, (mse, n) in enumerate(zip(mse_values, n_values)):
    print("The MSE for queries with rank",i,"is:")
    if n == 0:
        print("No test queries with that rank")
    else:
        print(mse / n)
        mse_general += mse / n
        n_general   += 1
    
print("The overall MSE is:")
print(mse_general/n_general)

The MSE for queries with rank 0 is:
0.0
The MSE for queries with rank 1 is:
1.0
The MSE for queries with rank 2 is:
1.2019270612786432
The MSE for queries with rank 3 is:
9.0
The MSE for queries with rank 4 is:
5.771845539184216
The overall MSE is:
3.3947545200925715


In [ ]:
x_te['Y_real'] = y_te
x_te['Y_pred'] = pred

In [ ]:
x_te[x_te["Query id"]==1].sort_values(by=['Y_pred'], ascending=False).head(20)

,Query id,f0,f1,f2,f3,Y_real,Y_pred
59,1,1.00000,0.671742,1.0,0,4,1.597533
48,1,0.57069,0.000000,1.0,0,2,0.911696
57,1,0.00000,0.000000,0.0,0,0,0.000000
21,1,0.00000,0.000000,0.0,0,0,0.000000
38,1,0.00000,0.000000,0.0,0,0,0.000000
20,1,0.00000,0.000000,0.0,0,0,0.000000
27,1,0.00000,0.000000,0.0,0,0,0.000000
43,1,0.00000,0.000000,0.0,0,0,0.000000
34,1,0.00000,0.000000,0.0,0,0,0.000000
60,1,0.00000,0.000000,0.0,0,0,0.000000


In [ ]:
x_te[x_te["Query id"]==2].sort_values(by=['Y_pred'], ascending=False).head(20)

,Query id,f0,f1,f2,f3,Y_real,Y_pred
107,2,0.472234,0.0,1.0,0,2,0.754409
110,2,0.472234,0.0,1.0,0,2,0.754409
126,2,0.472234,0.0,1.0,0,2,0.754409
75,2,0.472234,0.0,1.0,0,2,0.754409
116,2,0.472234,0.0,1.0,0,2,0.754409
99,2,0.472234,0.0,1.0,0,2,0.754409
124,2,0.000000,0.0,0.0,0,3,0.000000
109,2,0.000000,0.0,0.0,0,1,0.000000
76,2,0.000000,0.0,0.0,0,0,0.000000
111,2,0.000000,0.0,0.0,0,0,0.000000


In [ ]:
x_te[x_te["Query id"]==3].sort_values(by=['Y_pred'], ascending=False).head(20)

,Query id,f0,f1,f2,f3,Y_real,Y_pred
159,3,1.0,0.0,1.0,0,2,1.597533
181,3,1.0,0.0,1.0,0,4,1.597533
169,3,1.0,0.0,1.0,0,2,1.597533
190,3,0.0,0.0,0.0,0,1,0.000000
191,3,0.0,0.0,0.0,0,0,0.000000
175,3,0.0,0.0,0.0,0,1,0.000000
137,3,0.0,0.0,0.0,0,0,0.000000
183,3,0.0,0.0,0.0,0,1,0.000000
134,3,0.0,0.0,0.0,0,3,0.000000
184,3,0.0,0.0,0.0,0,1,0.000000
